In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import skimage as ski
from skimage.util import view_as_blocks as vb
import scipy

In [ ]:
def dct2(a):
    return scipy.fftpack.dct( scipy.fftpack.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return scipy.fftpack.idct( scipy.fftpack.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

In [ ]:
def DTC2_trans(w):
    K = w*w
    D = np.zeros((K,K))
    idx = 0
    for i in range(w):
        for j in range(w):
            x = np.zeros((w,w))            
            x[i,j]=1
            y = dct2(x)            
            D[idx,:] = y.reshape(1,K)
            idx = idx+1
    return D

In [ ]:
lamb = 0.1
window_size = 8

# load image
g = ski.io.imread("stradivari.png")/255.0
M = (g.shape[0]//window_size)*window_size
N = (g.shape[1]//window_size)*window_size
g = g[:M,:N]

# make DCT dictionary 
D=DTC2_trans(window_size)

# generate patches
patches = vb(g.copy(), (window_size, window_size))
BM,BN,_,_ = patches.shape
#print(patches.shape)

patches = patches.reshape(BM*BN,window_size*window_size)
#print(patches.shape)

# compute DCT coefficients
coeff = patches@D

# softshrinkage (all, but DC component)
coeff[:,1:] = np.maximum(0, np.abs(coeff[:,1:])-lamb)*np.sign(coeff[:,1:])

# reconstruct patches
patches = coeff@D.T

# reconstruct image
patches = patches.reshape(BM, BN,window_size, window_size)
u = patches.transpose(0,2,1,3).reshape(BM*window_size,BN*window_size)

plt.figure(1, figsize=(10,5))
plt.imshow(u, cmap="gray")

#ski.io.imsave("stradivari_compressed.png", u)